# Capture data from webcam

In [ ]:
import cv2
 
def generate_dataset():
    face_classifier = cv2.CascadeClassifier(r"D:/Deep_learning_project/Face_recognization/haarcascade_frontalface_default.xml")
    
    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)  # 1.3 is the scale factor, 5 is the minimum neighbor
         
        if len(faces) == 0:
            return None
        
        for (x, y, w, h) in faces:
            cropped_face = img[y:y+h, x:x+w]
        return cropped_face
    
    cap = cv2.VideoCapture(0)
    id = 2  # User ID
    img_id = 0  # Image ID for each capture
     
    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id += 1
            cropped_face = face_cropped(frame)
            flipped_face = cv2.flip(cropped_face, 1)  # Flip the cropped face horizontally
            resized_face = cv2.resize(flipped_face, (300, 300))  # Resize the face
            gray_face = cv2.cvtColor(resized_face, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
            
            # Define the file path where the image will be saved
            file_name_path = (r"D:/Deep_learning_project/Face_recognization/dataste/user." + str(id) + "." + str(img_id) + ".jpg")
            cv2.imwrite(file_name_path, gray_face)  # Save the face image
            
            # Add image ID as text on the image
            cv2.putText(gray_face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
             
            # Show the cropped face in a window
            cv2.imshow("Face_Cropped", gray_face)
             
        if cv2.waitKey(1) == 13 or int(img_id) == 200:  # Stop when 'Enter' key is pressed or 200 images are captured
            break
             
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed....")

generate_dataset()

# Frame generation from video

In [8]:
import cv2

def generate_faces_from_video(video_path, output_folder):
    face_classifier = cv2.CascadeClassifier(r"D:/Deep_learning_project/Face_recognization/haarcascade_frontalface_default.xml")

    cap = cv2.VideoCapture(video_path)
    
    count = 90 
    img_id = 90
    
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break

        count += 1
        if count == 200:
            break
        
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        faces = face_classifier.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            cropped_face = frame[y:y + h, x:x + w]
            
            resized_face = cv2.resize(cropped_face, (300, 300))

            gray_face = cv2.cvtColor(resized_face, cv2.COLOR_BGR2GRAY)

            img_id += 1
            file_name_path = f"{output_folder}/user.2.{img_id}.jpg"
            cv2.imwrite(file_name_path, gray_face)
 
            cv2.putText(gray_face, f"{img_id}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            cv2.imshow("Cropped Face", gray_face)
      
        if count % 5 == 0: 
            frame_filename = f"{output_folder}/Frame_{count}.jpg"
            cv2.imwrite(frame_filename, frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    print(f"Process completed. Frames and faces are saved in {output_folder}.")

video_path = r"D:/Deep_learning_project/Face_recognization/face_video_1.mp4"
output_folder = r"D:/Deep_learning_project/Face_recognization/dataste"

generate_faces_from_video(video_path, output_folder)


Process completed. Frames and faces are saved in D:/Deep_learning_project/Face_recognization/dataste.


# Train calssifier and save

In [9]:
import numpy as np
from PIL import Image
import os

In [10]:
def train_classifier(data_dir):
    path = [os.path.join(data_dir,f) for f in os.listdir(data_dir)]
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert("L")
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])

        faces.append(imageNp)
        ids.append(id)
    ids = np.array(ids)

    # Generate a face recognition model
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.train(faces,ids)
    recognizer.save("classifier.xml")
    print("Model trained successfully.")
train_classifier(r"D:/Deep_learning_project/Face_recognization/dataste")        

Model trained successfully.


In [ ]:
import cv2
import numpy as np
from PIL import Image
import os

def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
    
    for (x,y,w,h) in features:
        cv2.rectangle(img, (x,y), (x+w,y+h), color, 2 )
         
        id, pred = clf.predict(gray_img[y:y+h,x:x+w])
        confidence = int(100*(1-pred/300))
         
        if confidence>70:
            if id==1:
                cv2.putText(img, "Soudeep", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            if id==2:
                cv2.putText(img, "Fahim", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            cv2.putText(img, "Unknown", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 1, cv2.LINE_AA)
     
    return img
 
# loading classifier
faceCascade = cv2.CascadeClassifier(r"D:/Deep_learning_project/Face_recognization/haarcascade_frontalface_default.xml")
 
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read(r"D:/Deep_learning_project/Face_recognization/classifier.xml")
 
video_capture = cv2.VideoCapture(0)
 
while True:
    ret, img = video_capture.read()
    img = cv2.flip(img,1)
    img = draw_boundary(img, faceCascade, 1.3, 6, (255,255,255), "Face", clf)
    cv2.imshow("face Detection", img)
     
    if cv2.waitKey(1)==13:
        break
video_capture.release()
cv2.destroyAllWindows()